# Example / concept notebook for other MAST data identification use case

The use case itself is detailed [here](sky-viewer-other-data.md).  This notebook gives an idea of how this use case would work.  Some/all of the code may not work, but the goal is for the notebook (or a very similar notebook) to work once the development work has been completed.

* Note: throughout this example, the assumption is that Roman platform related functionality  lives in `astroquery.mast` this does not *need* to be the case - some of the functionality could live in some other helper package.  But for the sake of example we will assume all-in-astroquery.

In [ ]:
from astroquery.mast import Viewers, Observations, Catalogs
from astropy.coordinates import SkyCoord

import jwstdatamodels

In [ ]:
my_target = SkyCoord(1.2*u.deg, 3.4*u.deg, frame='some_bizzare_case_specific_frame')

In [ ]:
sky_viewer = Viewer.show_sky_viewer('sidecar') # pops out a new jupyterlab window on the side, just like jdaviz show('sidecar') does, but with the sky viewer instead of jdaviz.
sky_viewer.set_base_image('roman')  # this might be the default, but if not the user needs to be able to set it for this use case

In [ ]:
sky_viewer.center_on(my_target)

Now the user zooms in to an appropriate zoom level in the GUI but then realizes they want to come back to the same zoom_level later:

In [ ]:
sky_viewer.zoom_level  # in actual code this would print out the gui zoom level

In [ ]:
sky_viewer.zoom_level  = 10 # this would be whatever the user saw when they ran the cell above the *first time* (although on re-runs it will be something different of course.)

In [ ]:
roman_catalog = Catalogs.query_region(my_target, radius=10*u.arcmin, catalog='roman-sources')

... they do some filtering to produce a subset of the catalog they actually want as a boolean `msk` in the standard python/numpy way ...

my_sources_roman_catalog = roman_catalog[msk]

Now that the user has the sky viewer up they want and the relevant overplot Roman source catalogs, but they now need JWST observations.  There are now two possible paths, a notebook/Python-centric one, and a more UI-centric one.  The first is a requirement, while the latter may be a stretch goal.

## 1. notebook-centric querying

In [ ]:
jwst_obs = Observations.query_region(my_target, instrument_name='NIRSpec', radius=10*u.arcmin, instrument_config='NIRSpec/MSA')  #instrument_config may not yet exist but it could


## 2. UI/web-based searching (stretch goal)

Some users might feel more comfortable using a MAST web site-based approach rather than Python. In that case they would do something like the following:

In [ ]:
jwst_obs = Observations.search_in_mast()

This would then pop up a JWST missionMAST web view UI in a jupyterlab window, which would look just like the regular mission mast.  But when executed, instead of yielding the results page, it would close the panel and return the results to the above function call (i.e., `jwst_catalog` would be the astropy.table with the results).

Now the user needs to filter down to specific slits:

In [ ]:
jwst_products = Observations.get_data_products(jwst_obs)
spectrum_products = jwst_product[jwst_products['calib_level'] == 3] # for NIRSpec MSA, calib_level 3 is the extracted 1D spectra

sky_viewer.show_catalog(spectrum_products)  

`show_catalog` should be smart enough to recognize that the user wants the individual products plotted, and even if it's not in `spectrum_products`, it should know how to go get the coordinates to then show in the sky viewer.

At this point, the user sees both the Roman sources, and the JWST spectra on the same sky viewer view.  When they click on jwst targets, it then shows an identifier that maps to some column in the `spectrum_products` table:

In [ ]:
productFilenames_i_actually_want = ['jwxxxxxx_msa.fits', 'jwxxxxxy_msa.fits', ...]

desired_products = spectrum_products[np.in1d(spectrum_products['productFilename'], productFilenames_i_actually_want)]

spectra = Observations.download_products(desired_products, cache=True)

In the RSP the above download_products call should be instantaneous and instead just yield the necessary s3 identifiers.

In [ ]:
specviz = jdaviz.Specviz()
specviz.show('sidecar')

In [ ]:
for spectrum in spectra:
    specviz.load_spectra(spectrum)

Now the user can look through the spectrum in specviz and determine which ones have sufficient S/N and which need more follow up.  She annotates them here:

In [ ]:
msk = np.in1d(productFilenames_i_actually_want, spectra['productFilename'])
idx, _, _ = productFilenames_i_actually_want[msk]['coord'].match_to_catalog_sky(my_sources_roman_catalog['coord'])

to_observe = productFilenames_i_actually_want[msk][idx]